In [1]:
!pip install transformers rouge-score nltk blurr datasets py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 451 kB 86.5 MB/s 
     |████████████████████████████████| 65 kB 2.3 MB/s 
     |████████████████████████████████| 7.6 MB 84.4 MB/s 
     |████████████████████████████████| 182 kB 63.8 MB/s 
     |████████████████████████████████| 132 kB 74.5 MB/s 
     |████████████████████████████████| 132 kB 92.4 MB/s 
     |████████████████████████████████| 212 kB 96.4 MB/s 
     |████████████████████████████████| 127 kB 67.6 MB/s 
     |████████████████████████████████| 357 kB 97.9 MB/s 
     |████████████████████████████████| 50 kB 3.2 MB/s 
     |████████████████████████████████| 2.3 MB 85.9 MB/s 
     |████████████████████████████████| 139 kB 100.4 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████|

In [2]:
from transformers import BartTokenizer, AutoModelForSeq2SeqLM, \
Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, \
TFBartForConditionalGeneration, AutoTokenizer
import pandas as pd
import numpy as np
from google.colab import drive
from datasets import load_dataset, load_metric, load_from_disk
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import sklearn
import rouge_score
import nltk
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# @title Initialize 
## Home directory of drive
HOME = "/content/gdrive/My Drive/Colab Notebooks/"

In [4]:
# @title Read data
X_train = pd.read_csv(HOME+"Notebooks/Data/X_train.csv")
y_train = pd.read_csv(HOME+"Notebooks/Data/y_train.csv")
X_train['label'] = y_train['titles']
X_train.to_csv(HOME+"Notebooks/Data/train.csv", index=False)

X_train.sample(n=5000, random_state=923).to_csv(HOME+"Notebooks/Data/train_sampled.csv", index=False)
X_train.sample(n=2000, random_state=1218).to_csv(HOME+"Notebooks/Data/val_sampled.csv", index=False)

In [5]:
train = load_dataset('csv', data_files={'train': [HOME+"Notebooks/Data/train_sampled.csv"],
                                        'validation':[HOME+"Notebooks/Data/val_sampled.csv"]})

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4a5e720550e05383/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# @title Load model
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
# tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

 [Good source for fine-tuning BART](https://github.com/AldoF95/bart-chat-summarizer-finetuning/blob/main/Bart_large_xsum_fine_tuned_samsum.ipynb)

In [7]:
# @title Procressing the data
max_input = 512
max_target = 56

def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [x for x in data_to_process['abstracts']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['label'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [8]:
tokenize_data = train.map(preprocess_data, batched = True, remove_columns=['terms', 'abstracts', 'label'])

  0%|          | 0/5 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
metric = load_metric('rouge')

<ipython-input-9-10e4a614ebd6>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [10]:
#####################
# metrics
# compute rouge for evaluation 
#####################

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [11]:
args = Seq2SeqTrainingArguments(
    HOME+"Notebooks/Outputs/bart_finetuned_new", #save directory
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size= 2,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1
    )

collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
trainer = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [13]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 3750
  Number of trainable parameters = 406291456
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.632200,0.548273,51.087400,31.596100,45.744600,45.755500,14.988500
2,0.478900,0.510480,52.619100,32.215700,46.182700,46.129100,16.736000
3,0.395200,0.504065,52.794500,32.650000,46.769300,46.758600,16.083000


Saving model checkpoint to /content/gdrive/My Drive/Colab Notebooks/Notebooks/Outputs/bart_finetuned_new/checkpoint-500
Configuration saved in /content/gdrive/My Drive/Colab Notebooks/Notebooks/Outputs/bart_finetuned_new/checkpoint-500/config.json
Model weights saved in /content/gdrive/My Drive/Colab Notebooks/Notebooks/Outputs/bart_finetuned_new/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/gdrive/My Drive/Colab Notebooks/Notebooks/Outputs/bart_finetuned_new/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/gdrive/My Drive/Colab Notebooks/Notebooks/Outputs/bart_finetuned_new/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/gdrive/My Drive/Colab Notebooks/Notebooks/Outputs/bart_finetuned_new/checkpoint-1000
Configuration saved in /content/gdrive/My Drive/Colab Notebooks/Notebooks/Outputs/bart_finetuned_new/checkpoint-1000/config.json
Model weights saved in /content/gdrive/My Drive/Colab Notebooks/Notebooks/Ou

TrainOutput(global_step=3750, training_loss=0.7119291544596354, metrics={'train_runtime': 2757.1926, 'train_samples_per_second': 5.44, 'train_steps_per_second': 1.36, 'total_flos': 1.625328451584e+16, 'train_loss': 0.7119291544596354, 'epoch': 3.0})